In [1]:
import numpy as np
from keras.models import Model, load_model
from keras.layers import Dense, Input, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [2]:
X_train = np.load('../data/X_train.npy', mmap_mode = 'r')
y_train = np.load('../data/y_train_k.npy', mmap_mode = 'r')
X_val = np.load('../data/X_val.npy', mmap_mode = 'r')
y_val = np.load('../data/y_val_k.npy', mmap_mode = 'r')
mean = np.mean(X_train, axis = 0)
std = np.std(X_train, axis = 0)
std[std == 0] = 1
X_train = X_train - mean / std
X_val = X_val - mean / std

In [3]:
ins = Input(shape = (X_train.shape[1],))
x = Dense(4096)(ins)
x = Activation('relu')(x)
x = BatchNormalization()(x)
x = Dense(2048)(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)
x = Dense(1024)(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)
outs = Dense(y_train.shape[1], activation = 'softmax')(x)

model = Model(ins, outs)

In [4]:
opt = Adam()
model.compile(opt, 'categorical_crossentropy', metrics = ['acc'])
stopper = EarlyStopping(monitor='val_acc', patience = 3, mode='auto')
checker = ModelCheckpoint('../src/ensemble/reluModels/relu_test_best_8', monitor='val_acc', mode='auto')

In [5]:
model.fit(X_train, y_train,
          epochs=100, batch_size=32,
          validation_data=(X_val, y_val),
          callbacks = [stopper, checker])

Train on 3428511 samples, validate on 736105 samples
Epoch 1/100
3428511/3428511 [==============================] - 1757s - loss: 0.0210 - acc: 0.9971 - val_loss: 0.1086 - val_acc: 0.9925
Epoch 2/100
3428511/3428511 [==============================] - 1716s - loss: 0.0134 - acc: 0.9983 - val_loss: 0.2531 - val_acc: 0.9842
Epoch 3/100
3428511/3428511 [==============================] - 1716s - loss: 0.0121 - acc: 0.9985 - val_loss: 0.1440 - val_acc: 0.9901
Epoch 4/100
3428511/3428511 [==============================] - 1717s - loss: 0.0114 - acc: 0.9986 - val_loss: 0.1339 - val_acc: 0.9910
Epoch 5/100
3428511/3428511 [==============================] - 1717s - loss: 0.0110 - acc: 0.9987 - val_loss: 0.2567 - val_acc: 0.9834


In [2]:
model = load_model('../src/ensemble/relu_best.h5')  # Final model

In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 122)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              503808    
_________________________________________________________________
activation_1 (Activation)    (None, 4096)              0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 4096)              16384     
_________________________________________________________________
dense_2 (Dense)              (None, 2048)              8390656   
_________________________________________________________________
activation_2 (Activation)    (None, 2048)              0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 2048)              8192      
__________

In [4]:
from sklearn.metrics import accuracy_score

In [5]:
val_preds = model.predict(X_val)
int_preds_val = np.array([np.argmax(r) for r in val_preds])
int_y_val = np.array([np.argmax(r) for r in y_val])
val_acc = accuracy_score(int_y_val, int_preds_val)
print(val_acc)

0.998574931565


In [6]:
train_preds = model.predict(X_train)
int_preds_train = np.array([np.argmax(r) for r in train_preds])
int_y_train = np.array([np.argmax(r) for r in y_train])
train_acc = accuracy_score(int_y_train, int_preds_train)
print(train_acc)

0.998584808391


In [7]:
X_test = np.load('../data/X_test.npy', mmap_mode='r')
y_test = np.load('../data/y_test_k.npy', mmap_mode='r')

In [8]:
test_preds = model.predict(X_test)
int_preds = np.array([np.argmax(r) for r in test_preds])
int_y_test = np.array([np.argmax(r) for r in y_test])
test_acc = accuracy_score(int_y_test, int_preds)
print(test_acc)

0.992055218277


In [10]:
np.save('../data/preds/relu_val_preds', val_preds)
np.save('../data/preds/relu_train_preds', train_preds)
np.save('../data/preds/relu_test_preds', test_preds)